In [45]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv('diabetes.csv')

In [85]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [48]:
df.corr()['Outcome'] # Calculating the correlation of each feature with the target variable 'Outcome'

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [49]:
X=df.iloc[:,:-1].values # Separating features (X)
y=df.iloc[:,-1].values # and target variable (y)

In [50]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler() # Initializing the StandardScaler for feature scaling

In [51]:
X=scaler.fit_transform(X) # Scaling the features to standardize the data

In [8]:
X.shape

(768, 8)

In [52]:
from sklearn.model_selection import train_test_split
# Splitting the data into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [53]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout
# Importing necessary libraries for building and training a neural network

In [54]:
model=Sequential() # Building a basic sequential neural network model
model.add(Dense(32,activation='relu',input_dim=8)) # Adding a dense layer with 32 units and 'relu' activation
model.add(Dense(1,activation='sigmoid')) # Adding the output layer with 1 unit and 'sigmoid' activation for binary classification

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy']) # Compiling the model with Adam optimizer and binary crossentropy loss

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [55]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test)) # Training the initial model

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7189 - loss: 0.6181 - val_accuracy: 0.7468 - val_loss: 0.5915
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7536 - loss: 0.5690 - val_accuracy: 0.7857 - val_loss: 0.5595
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7416 - loss: 0.5440 - val_accuracy: 0.7987 - val_loss: 0.5393
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7435 - loss: 0.5267 - val_accuracy: 0.8052 - val_loss: 0.5236
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7653 - loss: 0.5030 - val_accuracy: 0.8117 - val_loss: 0.5154
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7526 - loss: 0.5096 - val_accuracy: 0.8052 - val_loss: 0.5060
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7677 - loss: 0.4933 - val_accuracy: 0.7987 - val_loss: 0.5004
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7952 - loss: 0.4629 - val_accuracy: 0.7987 - val_loss

In [13]:
#AUTOMATING THE PROCESS TO FIND OUT THE BEST WORKING PARAMETERS

In [14]:
# 1. SELECTING APPROPRIATE OPTIMIZER
# 2. SELECTING NO. OF NODES IN A LAYER
# 3. SELECTING NO. OF HIDDEN LAYERS

In [57]:
import keras_tuner as kt # Importing Keras Tuner library

In [58]:
# Defining a function to build the model with a tunable optimizer
def build_model(hp):

    model=Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta']) # Hyperparameter to tune: optimizer
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [59]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',   # objective is to maximise val accuracy
                     max_trials=5) # Initializing RandomSearch tuner to find the best optimizer

Reloading Tuner from ./untitled_project/tuner0.json


In [60]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))     #4 Trials and 5 Epochs # Performing the hyperparameter search for the best optimizer

In [24]:
# KNOWING BEST HYPER PARAMENTERS SO FAR

In [61]:
tuner.get_best_hyperparameters()[0].values # Getting the best hyperparameters found for the optimizer

{'optimizer': 'rmsprop'}

In [62]:
# FETCHING BEST MODEL AND THEN TRAINING IT

In [63]:
model=tuner.get_best_models(num_models=1)[0] # Retrieving the best model found by the tuner

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [64]:
model.summary() # Displaying the summary of the best model architecture

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [65]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test)) # Training the best model with the selected optimizer for more epochs

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7186 - loss: 0.5537 - val_accuracy: 0.7662 - val_loss: 0.5329
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7388 - loss: 0.5314 - val_accuracy: 0.7727 - val_loss: 0.5173
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7372 - loss: 0.5349 - val_accuracy: 0.7727 - val_loss: 0.5051
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7493 - loss: 0.5078 - val_accuracy: 0.7727 - val_loss: 0.4946
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7667 - loss: 0.4986 - val_accuracy: 0.7792 - val_loss: 0.4856
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7721 - loss: 0.4864 - val_accuracy: 0.7792 - val_loss: 0.4803
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7696 - loss: 0.4868 - val_accuracy: 0.7857 - val_loss: 0.4748
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8036 - loss: 0.4503 - val_accuracy:

In [ ]:
#Since Keras Tuner doesn’t allow you to change the hyperparameter space after creation, you need to create a new RandomSearch tuner each time you want to explore a new aspect.

In [66]:
# Defining a function to build the model with a tunable number of units in the first dense layer
def build_model(hp):
    model=Sequential()
    units=hp.Int('units',min_value=8,max_value=128,step=8)  #(lower_limit,upper_limit,step_size) # Hyperparameter to tune: number of units in the first dense layer
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [67]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     directory='mydir') # Initializing RandomSearch tuner to find the best number of units

Reloading Tuner from mydir/untitled_project/tuner0.json


In [68]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test)) # Performing the hyperparameter search for the number of units

In [69]:
tuner.get_best_hyperparameters()[0].values # Getting the best hyperparameters found for the number of units

{'units': 120}

In [70]:
model=tuner.get_best_models(num_models=1)[0] # Retrieving the best model with the selected number of units

In [71]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test)) # Training the best model with the selected number of units for more epochs

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7689 - loss: 0.5071 - val_accuracy: 0.7922 - val_loss: 0.4860
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7700 - loss: 0.4789 - val_accuracy: 0.7922 - val_loss: 0.4744
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7508 - loss: 0.4749 - val_accuracy: 0.7987 - val_loss: 0.4691
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7738 - loss: 0.4629 - val_accuracy: 0.7857 - val_loss: 0.4660
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7693 - loss: 0.4402 - val_accuracy: 0.7922 - val_loss: 0.4626
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7670 - loss: 0.4619 - val_accuracy: 0.7922 - val_loss: 0.4603
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7773 - loss: 0.4414 - val_accuracy: 0.7922 - val_loss: 0.4600
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7734 - loss: 0.4507 - val_accuracy: 0.79

In [72]:
# Defining a function to build the model with a tunable number of hidden layers and fixed units
def build_model(hp):
    model=Sequential()
    model.add(Dense(80,activation='relu',input_dim=8))

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)): # Hyperparameter to tune: number of hidden layers

      model.add(Dense(80,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [73]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=3,
                     directory='mydir',
                     project_name='num_layers') # Initializing RandomSearch tuner to find the best number of hidden layers

Reloading Tuner from mydir/num_layers/tuner0.json


In [74]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test)) # Performing the hyperparameter search for the number of hidden layers

In [75]:
tuner.get_best_hyperparameters()[0].values # Getting the best hyperparameters found for the number of hidden layers

{'num_layers': 10}

In [76]:
model=tuner.get_best_models(num_models=1)[0] # Retrieving the best model with the selected number of hidden layers

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [77]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test)) # Training the best model with the selected number of hidden layers for more epochs

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7721 - loss: 0.4449 - val_accuracy: 0.7987 - val_loss: 0.4936
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7818 - loss: 0.4762 - val_accuracy: 0.7403 - val_loss: 0.5124
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7815 - loss: 0.4481 - val_accuracy: 0.7338 - val_loss: 0.6925
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7669 - loss: 0.4940 - val_accuracy: 0.7857 - val_loss: 0.5105
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7913 - loss: 0.4491 - val_accuracy: 0.7922 - val_loss: 0.4992
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7937 - loss: 0.4250 - val_accuracy: 0.7662 - val_loss: 0.5271
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8220 - loss: 0.3947 - val_accuracy: 0.7662 - val_loss: 0.5308
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8207 - loss: 0.3917 - val_accuracy: 0.74

In [78]:
# Defining a function to build the model with tunable number of layers, units, activation, and dropout
def build_model(hp):
    model=Sequential()
    counter=0
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)): # Hyperparameter to tune: number of layers
        if counter==0:
            model.add(
                Dense(
                    hp.Int('units'+str(i),min_value=8,max_value=128,step=8), # Tunable units for the first layer
                    activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']), # Tunable activation for the first layer
                    input_dim=8
                    )
                )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))) # Tunable dropout for the first layer
        else:
           model.add(
                Dense(
                    hp.Int('units'+str(i),min_value=8,max_value=128,step=8), # Tunable units for subsequent layers
                    activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']), # Tunable activation for subsequent layers
                    input_dim=8
                    )
                )
           model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))) # Tunable dropout for subsequent layers
           counter+=1

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']), # Hyperparameter to tune: optimizer
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [79]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=3,
                     directory='mydir',
                     project_name='final1') # Initializing RandomSearch tuner to find the best combination of hyperparameters

Reloading Tuner from mydir/final1/tuner0.json


In [80]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test)) # Performing the hyperparameter search for the final model

In [81]:
tuner.get_best_hyperparameters()[0].values # Getting the best hyperparameters found for the final model

{'num_layers': 9,
 'units0': 24,
 'activation0': 'relu',
 'dropout0': 0.6,
 'optimizer': 'nadam',
 'units1': 88,
 'activation1': 'sigmoid',
 'dropout1': 0.6,
 'units2': 48,
 'activation2': 'tanh',
 'dropout2': 0.5,
 'units3': 48,
 'activation3': 'tanh',
 'dropout3': 0.2,
 'units4': 96,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 48,
 'activation5': 'sigmoid',
 'dropout5': 0.9,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1,
 'units8': 8,
 'activation8': 'relu',
 'dropout8': 0.1}

In [82]:
model=tuner.get_best_models(num_models=1)[0] # Retrieving the best model based on the final hyperparameter search

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 43 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [83]:
model.fit(X_train,y_train,epochs=106,initial_epoch=6,validation_data=(X_test,y_test)) # Training the final best model for more epochs

Epoch 7/106
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.4966 - loss: 0.7008 - val_accuracy: 0.6429 - val_loss: 0.6857
Epoch 8/106
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5579 - loss: 0.6931 - val_accuracy: 0.6429 - val_loss: 0.6827
Epoch 9/106
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6190 - loss: 0.6873 - val_accuracy: 0.6429 - val_loss: 0.6807
Epoch 10/106
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6584 - loss: 0.6789 - val_accuracy: 0.6429 - val_loss: 0.6783
Epoch 11/106
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6796 - loss: 0.6733 - val_accuracy: 0.6429 - val_loss: 0.6761
Epoch 12/106
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6668 - loss: 0.6744 - val_accuracy: 0.6429 - val_loss: 0.6738
Epoch 13/106
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6219 - loss: 0.6768 - val_accuracy: 0.6429 - val_loss: 0.6705
Epoch 14/106
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6687 - loss: 0.6666 - val_accuracy: 0.64